# **Preproccessing**

### Importing Asset

Dalam tahapan ini kita akan melakukan pengimportan terhadap hal hal yang memang diperlukan. Seperti kita akan mengimport library uyang akan dibutuhkan. Dan juga kita akn melakukan import pada datset kita. Selain itu kita akan melihat sedikit tentang isi datasetnya.

In [63]:
# import library
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import re
import string
import pandas as pd
import numpy as np

In [64]:
# mengambil dataset
df = pd.read_csv("dataset\INA_TweetsPPKM_Labeled_Pure.csv", sep="\t")

# menampilkan dataset
df.head()

,Date,User,Tweet,sentiment
0,2022-03-31 14:32:04+00:00,pikobar_jabar,Ketahui informasi pembagian #PPKM di wilayah J...,1
1,2022-03-31 09:26:00+00:00,inewsdotid,Tempat Ibadah di Wilayah PPKM Level 1 Boleh Be...,1
2,2022-03-31 05:02:34+00:00,vdvc_talk,"Juru bicara Satgas Covid-19, Wiku Adisasmito m...",1
3,2022-03-30 14:23:10+00:00,pikobar_jabar,Ketahui informasi pembagian #PPKM di wilayah J...,1
4,2022-03-30 11:28:57+00:00,tvOneNews,Kementerian Agama menerbitkan Surat Edaran Nom...,1


**0 = positif, 1 = netral, 2 = negatif**

lalu kita juga akan menelusuri berapa banyak data pada setiap label

In [65]:
# untuk menghitung banyaknya data pada setiap label
df["sentiment"].value_counts()

sentiment
1    17706
2     3980
0     1958
Name: count, dtype: int64

dari pengamatan di atas, kita dapat melihat bahwa tweet yang memiliki label netral jauh lebih banyak yang disusul dengan tweet negatif. Maka langkah yang kita akan ambil nantinya adalah dengan menyeimbangkan lagi untuk banyaknya data pada setiap label

### Feature Selection

dalam tahapan ini kita akan melakukan pengambilan fitur yang dibutuhkan saja. Untuk kasus text mining kita perlu komentarnya dan labelnya saja. Komentar sebagai x dan label sebagai y. Selain itu kita juga akan menyeimbangkan banyaknya data pada setiap label

In [66]:
# unuk menghilangkan kolom yang tidak diperlukan
df.drop(columns=["Date", "User"], inplace=True)

df.head()

,Tweet,sentiment
0,Ketahui informasi pembagian #PPKM di wilayah J...,1
1,Tempat Ibadah di Wilayah PPKM Level 1 Boleh Be...,1
2,"Juru bicara Satgas Covid-19, Wiku Adisasmito m...",1
3,Ketahui informasi pembagian #PPKM di wilayah J...,1
4,Kementerian Agama menerbitkan Surat Edaran Nom...,1


lalu kita akan melakukan pengecilan size. Agar seimbang kita tentukan banyaknya data pada setiap label adalh 1900, mengambil dari nilai terendah pada jenis label. Tujuan dari penyeimbangan data adalah untuk menghindari terjadinya overfitting pada label tertentu (cenderung menebak pada label yang banyak)

In [67]:
# untuk mengambil data 1900 dari masing2 jenis label
df = pd.concat([df[df.sentiment==0][:1900], df[df.sentiment==1][:1900], df[df.sentiment==2][:1900]])\
    .sample(frac=1, random_state=134).reset_index(drop=True)

df


,Tweet,sentiment
0,"Tak Terima Kota Malang Masuk PPKM Level 2, Wal...",1
1,"PPKM Dicabut, Booking Hotel Tetap Nihil https:...",1
2,Inilah Sistem Kerja Baru 2022 Bagi ASN Selama ...,1
3,JKW : Pak LBP nanti umumkan #PPKM #diperpanjan...,2
4,KABAR PANDEMI - Berpergian Sehat Pada Saat Pan...,1
...,...,...
5695,"#PPKM LEVEL 1000 yes , membagongkan",2
5696,"PPKM level 4 diperpanjang, sampai kapan? Nanti...",2
5697,"Mendagri Tito Karnavian mengatakan, PPKM Level...",1
5698,Program gagal kok yaa dilanjutin terus #PPKM #...,2


### Missing Value

Pada tahapan ini kita akan memeriksa apakah data kita terdapat yang kosong atau tidak. Tujuannya adalah untuk menghindari terjadinya error pada saat modeling algoritma

In [68]:
# mencari data kosong jenis Nan
df.isna().sum()

Tweet        0
sentiment    0
dtype: int64

In [69]:
# mencari data kosong jenis Null
df.isnull().sum()

Tweet        0
sentiment    0
dtype: int64

berhubungan tidak ada data yang kosong maka kita bisa langsung lanjut

### Duplicated Data

Selanjutnya kita juga akan melihat apakah ada data yang terduplikasi tujuannya adalah untuk menghindari overfitting. Model akan belajar hal yang sama secara berulang pada data yang terduplikasi sehingga model cenderung menghapal dan lebih mengingat data yang terduplikasi

In [70]:
# untuk menampilkan data yang terduplikasi
df[df.duplicated()]

,Tweet,sentiment
898,#PPKM,2
1258,Pemerintah Siapkan 3 Skenario Antisipasi Lonja...,1
1681,#semangat #ppkm,0
1691,Pemerintah Siapkan 3 Skenario Antisipasi Lonja...,1
1898,#ghostparade #ghostparadecomic #tidaktakuthant...,0
2108,#PPKM,2
2189,Pemerintah klo bikin acara rame2 di bolehin ti...,2
2286,#semangat #ppkm,0
2315,Hasil Pengeluaran BULLSEYE\nHari Ini : Selasa ...,1
2431,Pasti ada yang siap dan nggak siap dengan perp...,2


di atas merupakan beberapa data yang terduplikasi, maka kita perlu menghilangkan saja untuk data yang terduplikasi ini

In [71]:
# untuk menghilangkan data yang terduplikasi
df.drop_duplicates(inplace=True)

# untuk menampilkan data yang terduplikasi
df[df.duplicated]

,Tweet,sentiment


### Train Test Split

dalam tahapan ini kita akan membagi antara data train dengan data test. Tujuannya untuk kita dapat melakukan testing sehingga kita mengetahui performa dari model. Untuk sekarang kita menggunakan test size sebesar 0.2

In [72]:
# fungsi yang akan mengembalikan x, y yang sudah di split
def Train_Test_Split(x, y, test_size=0.2, random_seed=None):
    # banyaknya data
    n = len(x)

    # jika menentukan keacakan yang pasti
    if random_seed:
        np.random.seed(random_seed)

    # menghitung banyaknya datatest 
    test_size = int(test_size*n)

    # mengambil matrix random untuk menjadikan indices
    indices = np.random.permutation(n)
    train_indices, test_indices = indices[test_size:], indices[:test_size]

    # mengembalikan x_train, x_test, y_train, y_test
    return x.iloc[train_indices], x.iloc[test_indices], y.iloc[train_indices], y.iloc[test_size]

# menentukan x dan y nya
x = df["Tweet"]
y = df["sentiment"]

# melakukan split pada data kita
X_train, X_test, y_train, y_test = Train_Test_Split(x, y, random_seed=42)

# untuk menampilkan X_train
X_train

670     Finally, it's official guys! \nHarus Totalitas...
3509    Hai SohIB! Kapasitas penumpang #KRL Jabodetabe...
4408    Menteri Koordinator Bidang Perekonomian, Airla...
1622    37 kabupaten/kota di luar Jawa-Bali jalankan P...
2592    Ketahui informasi pembagian #PPKM di wilayah J...
                              ...                        
3796    Perpisahan tak membuat segalanya berakhir\nPin...
5227    PTM SD Masih 50 Persen PPKM Banyumas Turun Lev...
5264    Setiap kita yang 'usaha' hari ini sama-sama ke...
5430    Libur Aman dan Nyaman Tanpa Covid\n\nPemerinta...
860     Kasus COVID-19 di luar Jawa dan Bali masih rel...
Name: Tweet, Length: 4527, dtype: object

### Case Folding

Pada tahapan ini kita akan menghapus karater karakter yang tidak diperlukan dan tidak penting seperti tanda baca, hastag, tag, dan lain sebagainya.

In [81]:
# yang akan mengembalikan text yang sudah di casefolding
def case_folding(text):
    # untuk menghilangkan tag
    text = re.sub(r"@[A-Za-z0-9_]+", "", text)

    # untuk menghilangkan hastag
    text = re.sub(r"#\w+", "", text)

    # untuk menghilangkan angka
    text = re.sub(r"\d+", "", text)

    # untuk meghilangkan enter / newline
    text = re.sub(r"\n+", "", text)

    # untuk menghilangkan karakter RT 
    text = re.sub(r"RT[\s]+", "", text)

    # untuk menghilangkan link
    text = re.sub(r"https?://\S+", "", text)

    # untuk menghilangkan tanda baca
    text = text.translate(str.maketrans("", "", string.punctuation))

    # untuk menghilangkan emohji
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"
                               u"\U0001F300-\U0001F5FF"
                               u"\U0001F680-\U0001F6FF"
                               u"\U00010000-\U0010ffff"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r"", text)

    return text.lower()
    
    
# mengaplikasikannya untuk setiap dokumen
X_train = X_train.apply(case_folding)
X_train

670     finally its official guys harus totalitas tega...
3509    hai sohib kapasitas penumpang  jabodetabek dan...
4408    menteri koordinator bidang perekonomian airlan...
1622     kabupatenkota di luar jawabali jalankan ppkm ...
2592    ketahui informasi pembagian  di wilayah jabar ...
                              ...                        
3796    perpisahan tak membuat segalanya berakhirpintu...
5227    ptm sd masih  persen ppkm banyumas turun level...
5264    setiap kita yang usaha hari ini samasama kesus...
5430    libur aman dan nyaman tanpa covidpemerintah te...
860     kasus covid di luar jawa dan bali masih relati...
Name: Tweet, Length: 4527, dtype: object

mari kita cek untuk hasilnya

In [89]:
# sebelum di case folding
df.Tweet.loc[2160]

'Katanya #ppkm diperpanjang? Sampe berapa meter panjangnya tuh?? 🙄😏'

In [90]:
# setelah di case folding
X_train.loc[2160]

'katanya  diperpanjang sampe berapa meter panjangnya tuh '

### Tokenize

Pada tahapan ini kita akan melakuan pisahkn dokumen menjadi perkata. Hal ini difungsikan agar kita dapat menghitung bobot kata. Kata inila yang akan digunakan sebgai feature nantinya.

In [91]:
# melakukan pemisahan menjadi kata
def tokenize(text):
    return word_tokenize(text)

# mengimentasikan untuk setiap item
X_train  = X_train.apply(tokenize)
X_train

670     [finally, its, official, guys, harus, totalita...
3509    [hai, sohib, kapasitas, penumpang, jabodetabek...
4408    [menteri, koordinator, bidang, perekonomian, a...
1622    [kabupatenkota, di, luar, jawabali, jalankan, ...
2592    [ketahui, informasi, pembagian, di, wilayah, j...
                              ...                        
3796    [perpisahan, tak, membuat, segalanya, berakhir...
5227    [ptm, sd, masih, persen, ppkm, banyumas, turun...
5264    [setiap, kita, yang, usaha, hari, ini, samasam...
5430    [libur, aman, dan, nyaman, tanpa, covidpemerin...
860     [kasus, covid, di, luar, jawa, dan, bali, masi...
Name: Tweet, Length: 4527, dtype: object

### Stopwords

Tahapan stopwords adalah dimana kita membuang kata yang tidak begitu penting dan tidak bermakna secara independent. Karena sudah dipishkan menjadi perkata makan makna dai setiap kata haruslah independent. Misal kata hubung, kata sifat, dan lain lain 

In [95]:
# untuk mengambil kata yang tak bermakna
stopw = StopWordRemoverFactory().get_stop_words()

# fungsi yang menghilangkan stopword
def stopword(doc):
    return [text for text in doc if text not in stopw]

# mengimplementasikan pada setiap dokumen
X_train = X_train.apply(stopword)
X_train

670     [finally, its, official, guys, totalitas, tega...
3509    [hai, sohib, kapasitas, penumpang, jabodetabek...
4408    [menteri, koordinator, bidang, perekonomian, a...
1622    [kabupatenkota, luar, jawabali, jalankan, ppkm...
2592    [ketahui, informasi, pembagian, wilayah, jabar...
                              ...                        
3796    [perpisahan, tak, membuat, segalanya, berakhir...
5227      [ptm, sd, persen, ppkm, banyumas, turun, level]
5264    [usaha, hari, samasama, kesusahan, usaha, mend...
5430    [libur, aman, nyaman, covidpemerintah, menetap...
860     [kasus, covid, luar, jawa, bali, relatif, rend...
Name: Tweet, Length: 4527, dtype: object

### Stemming

Dalam tahapan ini kita akan mengubah kata yang memiliki imbuhan seperti -nya / me- / pe- menjadi kata aslinya (tanpa imbuhan). Hal ini difungsikan agar kata dalam setiap dokumen memiliki makna yang sama

In [104]:
# fungsi yang akan mengembalikan kata yang sudah di stem
stem = StemmerFactory().create_stemmer()

# fungsi yang akan mengembalikan dokumen yang sudah di stem
def stemming(doc):
    return [stem.stem(text) for text in doc]

# mengaplikasikan di setiap dokumen
X_train = X_train.apply(stemming)
# untuk berjaga jaga, kita mengcopynya
X_traincopy = X_train.copy()

dikarenakan prosesnya cukup lama maka kita perlu menynimpannya terlebih dahulu

In [105]:
# untuk disimpan terlebih dahulu
X_train.to_csv("stem2.csv", index=False)

X_train



670     [finally, its, official, guys, totalitas, tega...
3509    [hai, sohib, kapasitas, tumpang, jabodetabek, ...
4408    [menteri, koordinator, bidang, ekonomi, airlan...
1622    [kabupatenkota, luar, jawabali, jalan, ppkm, l...
2592    [tahu, informasi, bagi, wilayah, jabar, dasar,...
                              ...                        
3796    [pisah, tak, buat, segala, berakhirpintupintu,...
5227      [ptm, sd, persen, ppkm, banyumas, turun, level]
5264    [usaha, hari, samasama, susah, usaha, diskon, ...
5430    [libur, aman, nyaman, covidpemerintah, tetap, ...
860     [kasus, covid, luar, jawa, bal, relatif, renda...
Name: Tweet, Length: 4527, dtype: object

In [112]:
# mengupdate / memanggil ulang
X_train = pd.read_csv("stem2.csv")["Tweet"].apply(lambda x: x.replace("'", "").strip("][").split(","))

X_train

0       [finally,  its,  official,  guys,  totalitas, ...
1       [hai,  sohib,  kapasitas,  tumpang,  jabodetab...
2       [menteri,  koordinator,  bidang,  ekonomi,  ai...
3       [kabupatenkota,  luar,  jawabali,  jalan,  ppk...
4       [tahu,  informasi,  bagi,  wilayah,  jabar,  d...
                              ...                        
4522    [pisah,  tak,  buat,  segala,  berakhirpintupi...
4523    [ptm,  sd,  persen,  ppkm,  banyumas,  turun, ...
4524    [usaha,  hari,  samasama,  susah,  usaha,  dis...
4525    [libur,  aman,  nyaman,  covidpemerintah,  tet...
4526    [kasus,  covid,  luar,  jawa,  bal,  relatif, ...
Name: Tweet, Length: 4527, dtype: object

# **Pembobotan Kata**

### Analize Frequency Word

### Word Frequency on Document

### Term Frequency

### Inverse Document Frequency

### TF - IDF

### Feature Selection

# **Modeling**

### Create Class 

### Training

### Testing

### Conclusion